##### Package Installations including required model load, framework , embedding packages

In [1]:
! pip install -q transformers einops accelerate langchain bitsandbytes



In [2]:
! pip install --upgrade huggingface_hub

In [3]:
! pip install ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/b8/d4/ce436660098b2f456e2b8fdf76d4f33cbc3766c874c4aa2f772c7a5e943f/ipywidgets-8.1.0-py3-none-any.whl.metadata
  Using cached ipywidgets-8.1.0-py3-none-any.whl.metadata (2.4 kB)
  Obtaining dependency information for widgetsnbextension~=4.0.7 from https://files.pythonhosted.org/packages/8e/d4/d31b12ac0b87e8cc9fdb6ea1eb6596de405eaaa2f25606aaa755d0eebbc0/widgetsnbextension-4.0.8-py3-none-any.whl.metadata
  Using cached widgetsnbextension-4.0.8-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for jupyterlab-widgets~=3.0.7 from https://files.pythonhosted.org/packages/74/5e/2475ac62faf2e342b2bf20b8d8e375f49400ecb38f52e4e0a7557eb1cedb/jupyterlab_widgets-3.0.8-py3-none-any.whl.metadata
  Using cached jupyterlab_widgets-3.0.8-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.0-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.8-py3-none-any.whl (214 kB)
Us

### Login into Hugging face and use the access token from your account to get the Llama2 into the n/b

In [4]:
from huggingface_hub import login
login()

In [7]:
! pip install sentence_transformers

     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ------------- ------------------------ 30.7/86.0 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 86.0/86.0 kB 970.7 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/96/cf/a714a655266229b51eb2bda117f15275f12457887f165f3c1cc58ab502f1/scikit_learn-1.3.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/70/03/485f73046134400ea25d3cb178c5e6728f9b165f79d09638ecb44ee0e9b1/scipy-1.11.2-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ---------------------------------------- 59.1/59.1 kB ? eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     -------- -------------

In [1]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


In [2]:
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

#### Check for GPU being used or not

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available, using", torch.cuda.get_device_name(device))
else:
    print("No GPU detected, using CPU for computations.")
    device='cpu'

No GPU detected, using CPU for computations.


In [8]:
embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device':device},
    encode_kwargs={'device':device, 'batch_size':32}
)

In [10]:
docs = [

" Swin-Unet Existing medical image segmentation methods mainly rely on fully convolutional neural network (FCNN) with U-shaped structure [3,4,5]. ",
"The typical U-shaped network, U-Net [3], consists of a symmetric Encoder-Decoder with skip connections. ",
"In the encoder, a series of convolutional layers and continuous down-sampling layers are used to extract deep features with large receptive fields.",
"Then, the decoder up-samples the extracted deep features to the input resolution for pixel-level semantic prediction, and the high-resolution features", 
" of different scale from the encoder are fused with skip connections to alleviate the loss of spatial information caused by down-sampling. " , 
"With such an elegant structural design, U-Net has achieved great success in a variety of medical imaging applications." 

]


embeddings = embeddings_model.embed_documents(docs)

print(f"embeddings total {len(embeddings)} eith a dimensionality of {len(embeddings[0])}")

embeddings total 6 eith a dimensionality of 384


#### Build out your vectopr store

In [16]:
! pip install pinecone-client


  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/98/17/3675b83dca0a032d2750bf04fbfdf78a6e46fa3056eefc2574cdd14661d9/pinecone_client-2.2.2-py3-none-any.whl.metadata
  Using cached pinecone_client-2.2.2-py3-none-any.whl.metadata (7.8 kB)
  Obtaining dependency information for loguru>=0.5.0 from https://files.pythonhosted.org/packages/19/a9/4e91197b121a41c640367641a510fd9a05bb7a3259fc9678ee2976c8fd00/loguru-0.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for dnspython>=2.0.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
  Using cached dnspython-2.4.2-py3-none-any.whl.metadata (4.9 kB)
  Using cached win32_setctime-1.1.0-py3-none-any.whl (3.6 kB)
Using cached pinecone_client-2.2.2-py3-none-any.whl (179 kB)
Using cached dnspython-2.4.2-py3-none-any.whl (300 kB)
   ---------------------------------------- 0.0/61.4 kB

In [20]:
! pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [21]:
import os
import pinecone
import pinecone


from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)
pinecone.init(
    api_key = os.environ.get('PINECONE_API_KEY'),
    environment= os.environ.get('PINECONE_ENV')
)

#### Next step is to initialize the index

In [22]:
pinecone.list_indexes()

['askswinunet']

In [26]:
def delete_pinecone_index(index_name='all'):

    if index_name=='all':
        indexes = pinecone.list_indexes()
        print("Deleting allll indexes....")
        for index in indexes:
            pinecone.delete_index(index)
        print("ok")

    else:
        print(f'Deleting index {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('ok')
    

#### If you need to delete index : I have a basica free tier pinecone account that allows only 1 index at a given time

In [27]:
delete_pinecone_index()
pinecone.list_indexes()

Deleting allll indexes....
ok


[]

In [28]:
index_name = 'llama2-rag'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name}.....')
    pinecone.create_index(index_name,dimension=1536,metric="cosine", pods=1, pod_type="p1.x2")
    print('Done')
else:
    print(f'index {index_name} already exists')


Creating index llama2-rag.....
Done


In [29]:
## Connect to the index
index = pinecone.Index(index_name)
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

##### Model Load 

In [12]:
model = 'meta-llama/Llama-2-7b-hf'
tokernizer = AutoTokenizer.from_pretrained(model)



In [13]:
pipeline = transformers.pipeline(
    "text-generation", 
    model=model,
    tokernizer=tokernizer,
    torch_dtypr=torch.bfloat16,
    trust_remote_code=True,
    device_name="auto",
    max_length=100,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokernizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
llm = HuggingFacePipeline(pipeline=pipeline,model_kwargs={'temperature':0})

In [17]:
from langchain import PromptTemplate, LLMChain

template = """
        Write concise summary of the following text delimited by triple backquotes. Return your response in bullet points which covers the key points of the text. ''' {text}''' 
        BULLET POINT SuMMARY : 

 """

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
text = """
Swin-Unet
1 Technische Universit¨at M¨unchen, M¨unchen, Germany 2 Fudan University, Shanghai, China 3 Huawei Technologies, Shanghai, China
Abstract. In the past few years, convolutional neural networks (CNNs)
have achieved milestones in medical image analysis. Especially, the deep
neural networks based on U-shaped architecture and skip-connections
have been widely applied in a variety of medical image tasks. However, although CNN has achieved excellent performance, it cannot learn
global and long-range semantic information interaction well due to the
locality of convolution operation. In this paper, we propose Swin-Unet,
which is a Unet-like pure Transformer for medical image segmentation.
The tokenized image patches are fed into the Transformer-based Ushaped Encoder-Decoder architecture with skip-connections for localglobal semantic feature learning. Specifically, we use hierarchical Swin
Transformer with shifted windows as the encoder to extract context features. And a symmetric Swin Transformer-based decoder with patch expanding layer is designed to perform the up-sampling operation to restore the spatial resolution of the feature maps. Under the direct downsampling and up-sampling of the inputs and outputs by 4×, experiments on multi-organ and cardiac segmentation tasks demonstrate that
the pure Transformer-based U-shaped Encoder-Decoder network outperforms those methods with full-convolution or the combination of transformer and convolution. The codes and trained models will be publicly
available at https://github.com/HuCaoFighting/Swin-Unet.
1 Introduction
Benefiting from the development of deep learning, computer vision technology
has been widely used in medical image analysis. Image segmentation is an important part of medical image analysis. In particular, accurate and robust medical
image segmentation can play a cornerstone role in computer-aided diagnosis and
image-guided clinical surgery [1,2].
*Corresponding author † Work done as an intern in Huawei Technologies
arXiv:2105.05537v1 [eess.IV] 12 May 2021
2 Hu Cao et al.
Existing medical image segmentation methods mainly rely on fully convolutional neural network (FCNN) with U-shaped structure [3,4,5]. The typical
U-shaped network, U-Net [3], consists of a symmetric Encoder-Decoder with
skip connections. In the encoder, a series of convolutional layers and continuous down-sampling layers are used to extract deep features with large receptive
fields. Then, the decoder up-samples the extracted deep features to the input
resolution for pixel-level semantic prediction, and the high-resolution features
of different scale from the encoder are fused with skip connections to alleviate
the loss of spatial information caused by down-sampling. With such an elegant
structural design, U-Net has achieved great success in a variety of medical imaging applications. Following this technical route, many algorithms such as 3D
U-Net [6], Res-UNet [7], U-Net++ [8] and UNet3+ [9] have been developed for
image and volumetric segmentation of various medical imaging modalities. The
excellent performance of these FCNN-based methods in cardiac segmentation,
organ segmentation and lesion segmentation proves that CNN has a strong ability of learning discriminating features.
Currently, although the CNN-based methods have achieved excellent performance in the field of medical image segmentation, they still cannot fully meet
the strict requirements of medical applications for segmentation accuracy. Image
segmentation is still a challenge task in medical image analysis. Since the intrinsic locality of convolution operation, it is difficult for CNN-based approaches to
learn explicit global and long-range semantic information interaction [2]. Some
studies have tried to address this problem by using atrous convolutional layers [10,11], self-attention mechanisms [12,13], and image pyramids [14]. However,
these methods still have limitations in modeling long - range dependencies. Recently, inspired by Transformer’s great success in the nature language processing
(NLP) domain [15], researchers have tried to bring Transformer into the vision
domain [16]. In [17], vision transformer (ViT) is proposed to perform the image recognition task. Taking 2D image patches with positional embeddings as
inputs and pre-training on large dataset, ViT achieved comparable performance
with the CNN-based methods. Besides, data-efficient image transformer (DeiT)
is presented in [18], which indicates that Transformer can be trained on mid-size
datasets and that a more robust Transformer can be obtained by combining it
with the distillation method. In [19], a hierarchical Swin Transformer is developed. Take Swin Transformer as vision backbone, the authors of [19] achieved
state-of-the-art performance on Image classification, object detection and semantic segmentation. The success of ViT, DeiT and Swin Transformer in image
recognition task demonstrates the potential for Transformer to be applied in the
vision domain.
Motivated by the Swin Transformer’s [19] success, we propose Swin-Unet
to leverage the power of Transformer for 2D medical image segmentation in
this work. To our best knowledge, Swin-Unet is a first pure Transformer-based
U-shaped architecture that consists of encoder, bottleneck, decoder, and skip
connections. Encoder, bottleneck and decoder are all built based on Swin Transformer block [19]. The input medical images are split into non-overlapping image
Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation 3
patches. Each patch is treated as a token and fed into the Transformer-based
encoder to learn deep feature representations. The extracted context features are
then up-sampled by the decoder with patch expanding layer, and fused with the
multi-scale features from the encoder via skip connections, so as to restore the
spatial resolution of the feature maps and further perform segmentation prediction. Extensive experiments on multi-organ and cardiac segmentation datasets
indicate that the proposed method has excellent segmentation accuracy and robust generalization ability. Concretely, our contributions can be summarized as:
(1) Based on Swin Transformer block, we build a symmetric Encoder-Decoder
architecture with skip connections. In the encoder, self-attention from local to
global is realized; in the decoder, the global features are up-sampled to the input resolution for corresponding pixel-level segmentation prediction. (2) A patch
expanding layer is developed to achieve up-sampling and feature dimension increase without using convolution or interpolation operation. (3) It is found in
the experiment that skip connection is also effective for Transformer, so a pure
Transformer-based U-shaped Encoder-Decoder architecture with skip connection
is finally constructed, named Swin-Unet.
2 Related work
CNN-based methods : Early medical image segmentation methods are mainly
contour-based and traditional machine learning-based algorithms [20,21]. With
the development of deep CNN, U-Net is proposed in [3] for medical image segmentation. Due to the simplicity and superior performance of the U-shaped
structure, various Unet-like methods are constantly emerging, such as Res-UNet [7],
Dense-UNet [22], U-Net++ [8] and UNet3+ [9]. And it is also introduced into
the field of 3D medical image segmentation, such as 3D-Unet [6] and V-Net [23].
At present, CNN-based methods have achieved tremendous success in the field
of medical image segmentation due to its powerful representation ability.
Vision transformers : Transformer was first proposed for the machine translation task in [15]. In the NLP domain, the Transformer-based methods have
achieved the state-of-the-art performance in various tasks [24]. Driven by Transformer’s success, the researchers introduced a pioneering vision transformer (ViT)
in [17], which achieved the impressive speed-accuracy trade-off on image recognition task. Compared with CNN-based methods, the drawback of ViT is that it
requires pre-training on its own large dataset. To alleviate the difficulty in training ViT, Deit [18] describes several training strategies that allow ViT to train
well on ImageNet. Recently, several excellent works have been done baed on
ViT [25,26,19]. It is worth mentioning that an efficient and effective hierarchical
vision Transformer, called Swin Transformer, is proposed as a vision backbone
in [19]. Based on the shifted windows mechanism, Swin Transformer achieved
the state-of-the-art performance on various vision tasks including image classification, object detection and semantic segmentation. In this work, we attempt to
use Swin Transformer block as basic unit to build a U-shaped Encoder-Decoder
4 Hu Cao et al.
architecture with skip connections for medical image segmentation, thus providing a benchmark comparison for the development of Transformer in the medical
image field.
Self-attention/Transformer to complement CNNs : In recent years, researchers have tried to introduce self-attention mechanism into CNN to improve
the performance of the network [13]. In [12], the skip connections with additive
attention gate are integrated in U-shaped architecture to perform medical image
segmentation. However, this is still the CNN-based method. Currently, some efforts are being made to combine CNN and Transformer to break the dominance
of CNNs in medical image segmentation [2,27,1]. In [2], the authors combined
Transformer with CNN to constitute a strong encoder for 2D medical image segmentation. Similar to [2], [27] and [28] use the complementarity of Transformer
and CNN to improve the segmentation capability of the model. Currently, various combinations of Transformer with CNN are applied in multi-modal brain
tumor segmentation [29] and 3D medical image segmentation [1,30]. Different
from the above methods, we try to explore the application potential of pure
Transformer in medical image segmentation.
3 Method
3.1 Architecture overview
The overall architecture of the proposed Swin-Unet is presented in Figure. 1.
Swin-Unet consists of encoder, bottleneck, decoder and skip connections. The
basic unit of Swin-Unet is Swin Transformer block [19]. For the encoder, to
transform the inputs into sequence embeddings, the medical images are split into
non-overlapping patches with patch size of 4 × 4. By such partition approach,
the feature dimension of each patch becomes to 4 × 4 × 3 = 48. Furthermore, a
linear embedding layer is applied to projected feature dimension into arbitrary
dimension (represented as C). The transformed patch tokens pass through several
Swin Transformer blocks and patch merging layers to generate the hierarchical
feature representations. Specifically, patch merging layer is responsible for downsampling and increasing dimension, and Swin Transformer block is responsible
for feature representation learning. Inspired by U-Net [3], we design a symmetric transformer-based decoder. The decoder is composed of Swin Transformer
block and patch expanding layer. The extracted context features are fused with
multiscale features from encoder via skip connections to complement the loss
of spatial information caused by down-sampling. In contrast to patch merging
layer, a patch expanding layer is specially designed to perform up-sampling. The
patch expanding layer reshapes feature maps of adjacent dimensions into a large
feature maps with 2× up-sampling of resolution. In the end, the last patch expanding layer is used to perform 4× up-sampling to restore the resolution of the
feature maps to the input resolution (W ×H), and then a linear projection layer
is applied on these up-sampled features to output the pixel-level segmentation
predictions. We would elaborate each block in the following
Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation 5
Fig. 1. The architecture of Swin-Unet, which is composed of encoder, bottleneck, decoder and skip connections. Encoder, bottleneck and decoder are all constructed based
on swin transformer block.
3.2 Swin Transformer block
Different from the conventional multi-head self attention (MSA) module, swin
transformer block [19] is constructed based on shifted windows. In Figure. 2,
two consecutive swin transformer blocks are presented. Each swin transformer
block is composed of LayerNorm (LN) layer, multi-head self attention module,
residual connection and 2-layer MLP with GELU non-linearity. The windowbased multi-head self attention (W-MSA) module and the shifted window-based
multi-head self attention (SW-MSA) module are applied in the two successive
6 Hu Cao et al.
Fig. 2. Swin transformer block.
transformer blocks, respectively. Based on such window partitioning mechanism,
continuous swin transformer blocks can be formulated as:

"""

##### Run Code 

In [ ]:
llm_chain.run(text)